# [Module 2.2] 세이지 메이커 인퍼런스

본 워크샵의 모든 노트북은 `conda_python3` 추가 패키지를 설치하고 모두 이 커널 에서 작업 합니다.

- 1. 배포 준비
- 2. 로컬 앤드포인트 생성
- 3. 로컬 추론



---    
    

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

이전 노트북에서 인퍼런스 테스트를 완료한 티펙트를 가져옵니다.

In [2]:
%store -r artifact_path
%store -r bucket
%store -r prefix

# 1. 배포 준비

In [3]:
print("artifact_path: ", artifact_path)

artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-05-17-11-59-58-456/output/model.tar.gz


In [4]:
import sagemaker

role = sagemaker.get_execution_role()

## 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [5]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

In [6]:
class Params:
    def __init__(self):
        self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        self.lr = 0.001
        self.top_k = 10
        self.out = True
        self.gpu = "0"
                        
args = Params()
print("# of epochs: ", args.epochs)

import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



# of epochs:  1


In [7]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}

# predict(inf_model, payload, top_k=10)

# 2. 엔드포인트 생성
- 이 과정은 세이지 메이커 엔드포인트를 생성합니다.



In [8]:
import os
import time
from sagemaker.pytorch.model import PyTorchModel

instance_type = 'ml.p2.xlarge'

In [9]:
%%time 

endpoint_name = "sm-ncf-{}".format(int(time.time()))


sm_pytorch_model = PyTorchModel(model_data=artifact_path,
                                   role=role,
                                   entry_point='inference.py',
                                   source_dir = 'src',
                                   framework_version='1.8.1',
                                   py_version='py3',
                                   model_server_workers=1,
                                  )

sm_predictor = sm_pytorch_model.deploy(instance_type= instance_type, 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True,
                        )

------------!CPU times: user 830 ms, sys: 73 ms, total: 903 ms
Wall time: 6min 3s


In [14]:
from sagemaker.serializers import JSONSerializer
sm_predictor.serializer = JSONSerializer('application/json')

# 엔드 포인트 추론

In [15]:
type(payload)

dict

In [16]:
sm_predictor.predict(payload)

array([ 25, 273, 128, 881, 464, 175, 864,  58, 340, 174])

In [18]:
import boto3

if instance_type == 'local_gpu':
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
else:
    runtime_client = boto3.Session().client('sagemaker-runtime')



In [21]:
import json
from inference_utils import invoke_endpoint
# https://stackoverflow.com/questions/68072684/call-sagemaker-endpoint-invoke-endpoint-in-lambda-function-question-for-request

# data = json.loads(json.dumps(payload))
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

--- 0.018736839294433594 seconds ---
result:  ['[25, 273, 128, 881, 464, 175, 864, 58, 340, 174]']


# Clean-up

위의 엔드포인트를 삭제 합니다.

In [23]:
from inference_utils import delete_endpoint

# client = sagemaker.local.LocalSagemakerClient()
client = boto3.Session().client('sagemaker')
delete_endpoint(client, endpoint_name)

--- Deleted model: pytorch-inference-2022-05-21-07-14-49-329
--- Deleted endpoint: sm-ncf-1653117288
--- Deleted endpoint_config: sm-ncf-1653117288
